In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from scipy.stats import linregress

In [ ]:
fish_info = pd.read_excel("fish_info.xlsx")
fish_info['Fish ID'] = fish_info['Fish ID'].astype('str')
fish_info['Carcass Status'] = fish_info['Carcass Status'].astype('int8')
fish_info.set_index('Fish ID', inplace=True)
fish_info

In [ ]:
data = pd.read_excel("matlab_XLR.xlsx", sheet_name="Results Summary")
vn = data.columns
lt_track = np.zeros(data.shape, dtype = "bool")
for i in np.arange(3, len(vn)):
    lt_track[:, i] = ~(data[vn[i]].str.removeprefix("<") == data[vn[i]])
    temp = data[vn[i]].str.removeprefix("<")
    data[vn[i]] = temp
    
data

In [ ]:
data = data.loc[data.Units.str.contains('wwt'), :]
liver_id = data.columns.str.contains('liver', case = False)
muscle_id = data.columns.str.contains('MT', case = False)

# ploting
col_liver = data.columns[liver_id].str.removeprefix('White Sturgeon (WS #').str.removesuffix(') Liver')
col_muscle = data.columns[muscle_id].str.removeprefix('White Sturgeon (WS #').str.removesuffix(') MT')
x_liver = fish_info.loc[col_liver, 'Carcass Status'].to_numpy('int')
x_muscle = fish_info.loc[col_muscle, 'Carcass Status'].to_numpy('int')


for element in data['Analyte']:
    plt.figure(figsize=(15, 6), facecolor='white')
    
    ## liver
    y = data.loc[data.Analyte == element, liver_id].to_numpy('float32')
    plt.subplot(1,2,1)
    plt.scatter(x_liver, y)
    plt.title('Liver')
    plt.xlabel('carcass status')
    plt.ylabel('mg/kg wwt')

    slope, intercept, r_value, p_value, std_err = linregress(x_liver, y)
    plt.plot(x_liver,slope * x_liver + intercept, label = 'linereg', color = 'green')
    plt.annotate(f' y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (np.mean(plt.xlim())-np.diff(plt.xlim())*.13, plt.ylim()[1]-np.diff(plt.ylim())*.14))

    # muscle
    y = data.loc[data.Analyte == element, muscle_id].to_numpy('float32')
    plt.subplot(1,2,2)
    plt.scatter(x_muscle, y)
    plt.title('Muscle')
    plt.xlabel('carcass status')
    plt.ylabel('mg/kg wwt')
    
    slope, intercept, r_value, p_value, std_err = linregress(x_muscle, y)
    plt.plot(x_muscle,slope * x_muscle + intercept, label = 'linereg', color = 'green')
    plt.annotate(f' y = {slope:.2f} * x + {intercept:.2f}'+
                    "\n"+f'p_value:{p_value:.2f}' + "\n"+f'r_squared:{r_value ** 2:.2f}',
                    (np.mean(plt.xlim())-np.diff(plt.xlim())*.13, plt.ylim()[1]-np.diff(plt.ylim())*.14))



    plt.suptitle(f'element: {element}');
    
    if not os.path.exists('./pics/carcass/'):
        os.mkdir('./pics/carcass/')
    plt.savefig(f'./pics/carcass/{element}.jpg')
    